In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df_new = pd.read_csv(r'Watcho_DA_Subs_new_Mar12', sep='\t', encoding='utf-8', index_col=0)

In [ ]:
df_new.head()

In [ ]:
# Summary statistics
print("\nSummary statistics:")
df_new.describe()

In [ ]:
def missing_values_summary(df):
    total = df.isnull().sum()
    percent = (df.isnull().sum() / df.isnull().count() * 100)
    missing_data = pd.DataFrame({'Total Missing': total, 'Percentage': percent})
    return missing_data

# Use the function and display the summary
missing_summary = missing_values_summary(df_new)
print("\nMissing values summary:")
print(missing_summary)

In [ ]:
#########  creating aggregated columns for last 3 months
df_new['notification_sent_last_3_months'] = df_new['notification_sent_DEC_24'].fillna(0)+ df_new['notification_sent_JAN_25'].fillna(0)+df_new['notification_sent_sms_FEB_25'].fillna(0)
df_new['notification_sent_sms_3_months'] = df_new['notification_sent_sms_DEC_24'].fillna(0)+df_new['notification_sent_sms_JAN_25'].fillna(0)+df_new['notification_sent_sms_FEB_25'].fillna(0)
df_new['notification_clicked_3_months'] = df_new['notification_clicked_DEC_24'].fillna(0)+df_new['notification_clicked_JAN_25'].fillna(0)+df_new['notification_clicked_FEB_25'].fillna(0)
df_new['notification_sent_watsapp_3_months'] = df_new['notification_sent_watsapp_DEC_24'].fillna(0)+df_new['notification_sent_watsapp_JAN_25'].fillna(0)+df_new['notification_sent_watsapp_FEB_25'].fillna(0)
df_new['active_days_3_months'] = df_new['ActiveDays_Dec24'].fillna(0)+df_new['ActiveDays_Jan25'].fillna(0)+df_new['ActiveDays_Feb25'].fillna(0)
df_new['revenue_last_3_months'] = df_new['Revenue_Dec24'].fillna(0)+df_new['Revenue_Jan25'].fillna(0)+df_new['Revenue_Feb25'].fillna(0)

In [ ]:
df_new = df_new.drop(columns=['notification_sent_DEC_24','notification_sent_JAN_25','notification_sent_FEB_25',
                              'notification_sent_sms_DEC_24','notification_sent_sms_JAN_25','notification_sent_sms_FEB_25',
                              'notification_clicked_DEC_24','notification_clicked_JAN_25','notification_clicked_FEB_25',
                             'notification_sent_watsapp_DEC_24','notification_sent_watsapp_JAN_25','notification_sent_watsapp_FEB_25',
                             'ActiveDays_Dec24','ActiveDays_Jan25','ActiveDays_Feb25',
                             'Revenue_Dec24','Revenue_Jan25','Revenue_Feb25'])

In [ ]:
df_new

In [ ]:
df_new.groupby('customercategory').agg({'no_of_times_viewed_watcho_content_in_last_3_months':'sum'}).reset_index()

In [ ]:
df_new.groupby('customercategory').agg({'notification_clicked_3_months':'sum'}).reset_index()

In [ ]:
# Use the function and display the summary
missing_summary = missing_values_summary(df_new)
print("\nMissing values summary:")
print(missing_summary)

In [ ]:
df = df_new

In [ ]:
### Creating current_packageid_Category from current_packageid

conditions = [
    (df['current_packageid'] > 0) & (df['current_packageid'] <= 50000),
    (df['current_packageid'] > 50000) & (df['current_packageid'] <= 100000),
    (df['current_packageid'] > 100000) & (df['current_packageid'] <= 150000),
    (df['current_packageid'] > 150000) & (df['current_packageid'] <= 200000)
]


values = ['a. 0 - 50k', 'b. 50k - 1Lac.', 'c.1Lac. - 1.5Lac.','d.1.5Lac. - 2Lac.']

# Create a new column 'B' based on the conditions
df['current_packageid_Category'] = pd.cut(df['current_packageid'], bins=[0,50000, 100000, 150000, 200000], labels=values, right=False)

df.head()

In [ ]:
# checking if we missed any packageid value while bining i.e. no category assigned 
df[df['current_packageid_Category'].isnull() == True]

In [ ]:
df

In [ ]:
missing_summary = missing_values_summary(df)
print("\nMissing values summary:")
print(missing_summary)

In [ ]:
## columns with NAN that can be replaced with 0

columns_to_replace = ['Is_hotstar_subscriber','Is_sonyliv_subscriber','Is_zee5_subscriber','Is_hungama_subscriber','Is_fancode_subscriber',
                      'Is_chaupal_subscriber','active_days_3_months','last_price_point','no_of_renewals','Days_since_last_renewal',
                      'no_of_times_viewed_watcho_content_in_last_3_months','notification_clicked_3_months',
                      'no_of_times_viewed_clicked_content_in_last_3_months_HotStar','no_of_times_viewed_clicked_content_in_last_3_months_Zee5']


df.loc[:, columns_to_replace] = df.loc[:, columns_to_replace].where(pd.notna(df.loc[:, columns_to_replace]), 0)

In [ ]:
## Columns with NAN that can be replaced with Mode

# List of columns to fill
columns_to_fill = ['RevenueRenewal','Current_Debit_mode']

# Apply the filling operation only to the selected columns
df_clean = df.copy()
df_clean[columns_to_fill] = df_clean[columns_to_fill].apply(lambda x: x.fillna(x.value_counts().index[0]))


In [ ]:
pd.set_option('display.max_columns', None)
df

In [ ]:
missing_summary = missing_values_summary(df_clean)
print("\nMissing values summary:")
print(missing_summary)

In [ ]:
df_clean['DTH_Status'].value_counts()

In [ ]:
df_clean

In [ ]:
categorical_columns = ['RevenueRenewal','customercategory','Current_Debit_mode','current_packageid_Category','Smart_Plus_Customer','DTH_Status',
                       'Is_hotstar_subscriber','Is_sonyliv_subscriber','Is_zee5_subscriber','Is_hungama_subscriber','Is_fancode_subscriber',
                       'Is_chaupal_subscriber','is_first_time_da']

cat_col_requiring_cat = ['RevenueRenewal','customercategory','Current_Debit_mode','current_packageid_Category','Smart_Plus_Customer','DTH_Status']

numeric_columns = ['Days_since_last_renewal','AON','no_of_renewals','last_price_point','no_of_times_viewed_watcho_content_in_last_3_months',
                   'notification_clicked_3_months','notification_sent_last_3_months','notification_sent_sms_3_months',
                   'notification_sent_watsapp_3_months','notification_clicked_3_months','notification_sent_watsapp_3_months',
                  'no_of_times_viewed_clicked_content_in_last_3_months_HotStar','no_of_times_viewed_clicked_content_in_last_3_months_Zee5']        

In [ ]:
categorical_columns

In [ ]:
numeric_columns

In [ ]:
# Create data profile for numeric columns
for col in numeric_columns:
    print(f"Data Profile for Numeric Column: {col}")
    print(df_clean[col].describe())
    plt.figure(figsize=(10, 5))
    sns.histplot(df_new[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.show()

In [ ]:
# Plotting categorical columns
for col in categorical_columns:
    plt.figure(figsize=(10, 5))
    sns.countplot(x=col, data=df_clean)
    plt.title(f'Distribution of {col}')
    plt.show()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = df_clean[numeric_columns].corr()

print("Correlation Matrix:")
print(correlation_matrix) 

In [ ]:
### Outlier management for Numerical columns 
###outliers_columns = ['no_of_renewals','last_price_point']

def Capping_value(df, Cols_operation):
    for col, operation in Cols_operation.items():
        df[col] = df[col].apply(operation)
    return df        
    

Cols_operation = {
    'no_of_renewals': lambda x: 0 if x < 0 else (20 if x >20 else x),
    'last_price_point': lambda x: 0 if x < 0 else(2000 if x > 2000 else x)
    }


df = Capping_value(df_clean,Cols_operation)

In [ ]:
df = df.drop(columns=['current_packageid','Status_on_30Nov','activationdate','Last_Renewal_date','Is_livetv_subscriber',
                     'no_of_times_viewed_watcho_content_in_last_3_months_HotStar','no_of_times_viewed_watcho_content_in_last_3_months_Zee5'])

In [ ]:
df

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [ ]:
## Normalisation
scaler = MinMaxScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

In [ ]:
# One hOt encoding
df = pd.get_dummies(df,columns = cat_col_requiring_cat, drop_first = True)
df
 

In [ ]:
df.to_csv("Preprocessed Data.csv", encoding = "utf-8")

In [ ]:
df = df.drop('smsid', axis =1)

In [ ]:
df['Status_As_on_28Feb'] = df['Status_As_on_28Feb'].apply(lambda x: 0 if x == 'Deactive' else 1)

In [ ]:
X = df.drop('Status_As_on_28Feb', axis=1)
y = df['Status_As_on_28Feb']

In [ ]:
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize model
model = LogisticRegression()

# Train model
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f'ROC-AUC Score: {roc_auc}')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
 
# Get the predicted probabilities for the positive class (class 1) for both models
y_probs_lr = model.predict_proba(X_test)[:, 1]

 
# Compute the ROC curve for Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_probs_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)
 

# Plot the ROC curve
plt.figure(figsize=(8, 6))
 
# Plot for Logistic Regression
plt.plot(fpr_lr, tpr_lr, color='b', label=f'Logistic Regression (AUC = {roc_auc_lr:.2f})')
 
# Plot for Random Forest
# plt.plot(fpr_rf, tpr_rf, color='r', label=f'Random Forest (AUC = {roc_auc_rf:.2f})')
 
# Random classifier line
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
 
# Labels and title
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
 
# Show plot
plt.show()
 
# Print the AUC values
print(f"Logistic Regression AUC: {roc_auc_lr:.2f}")
# print(f"Random Forest AUC: {roc_auc_rf:.2f}")

In [ ]:
####### working with adjusted thresholds 

y_probs = model.predict_proba(X_test)[:, 1]


threshold = 0.1

y_pred_adjusted = (y_probs >= threshold).astype(int)

precision = precision_score(y_test, y_pred_adjusted, average='binary')
recall = recall_score(y_test, y_pred_adjusted, average='binary')
f1 = f1_score(y_test, y_pred_adjusted, average='binary')

print(f'Threshold: {threshold}')
print(f'Adjusted Precision: {precision}')
print(f'Adjusted Recall: {recall}')
print(f'Adjusted F1-Score: {f1}')



####Logistic Regression with balanced weights

In [ ]:
# Initialize model
model = LogisticRegression(class_weight = 'balanced')

# Train model
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
 
# Get the predicted probabilities for the positive class (class 1) for both models
y_probs_lr = model.predict_proba(X_test)[:, 1]

 
# Compute the ROC curve for Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_probs_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)
 

# Plot the ROC curve
plt.figure(figsize=(8, 6))
 
# Plot for Logistic Regression
plt.plot(fpr_lr, tpr_lr, color='b', label=f'Logistic Regression (AUC = {roc_auc_lr:.2f})')
 
# Plot for Random Forest
# plt.plot(fpr_rf, tpr_rf, color='r', label=f'Random Forest (AUC = {roc_auc_rf:.2f})')
 
# Random classifier line
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
 
# Labels and title
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
 
# Show plot
plt.show()
 
# Print the AUC values
print(f"Logistic Regression AUC: {roc_auc_lr:.2f}")
# print(f"Random Forest AUC: {roc_auc_rf:.2f}")